In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import mysql.connector

db = mysql.connector.connect(host = "localhost",
                            username = "root",
                            password = "Tasbiha@1234",
                            database = "ecommerce")

cur = db.cursor()

# 1. List all unique cities where customers are located.

In [5]:
query = """
SELECT DISTINCT
    customer_city
FROM
    customers AS unique_citiesa;
"""
cur.execute(query)
data = cur.fetchall()
df = pd.DataFrame(data)
df.head()

,0
0,franca
1,sao bernardo do campo
2,sao paulo
3,mogi das cruzes
4,campinas


## **2. Count the number of orders placed in 2017.**

In [4]:
query = """
SELECT 
    count(order_id)
FROM
    orders
WHERE
    YEAR(order_purchase_timestamp) = '2017';
"""

cur.execute(query)
data = cur.fetchall()
"total order placed in year 2017 are: ", data[0][0]

('total order placed in year 2017 are: ', 45101)

# 3. Find the total sales per category.